## Libraries

In [15]:
import pandas as pd 
import dateparser
import re

## Exploration analysis

In [7]:
data = pd.read_csv("text_data.csv")
data.head()

,url,date,article
0,https://www.la-prensa.com.mx/policiaca/acciden...,jueves 31 de octubre de 2024,biker sobrevive a peligrosa maniobra en la mex...
1,https://www.la-prensa.com.mx/policiaca/arresta...,lunes 28 de octubre de 2024,arrestan a mujer en la vcarranza acusada de ma...
2,https://www.la-prensa.com.mx/policiaca/detiene...,martes 5 de noviembre de 2024,detienen a catorce personas luego de tres dias...
3,https://www.la-prensa.com.mx/policiaca/vincula...,lunes 28 de octubre de 2024,"vinculan a proceso a ricardo ""n"" ex funcionari..."
4,https://www.la-prensa.com.mx/policiaca/caen-do...,martes 5 de noviembre de 2024,caen dos con tarjetas de credito presuntamente...


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12482 entries, 0 to 12481
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      12482 non-null  object
 1   date     12482 non-null  object
 2   article  12482 non-null  object
dtypes: object(3)
memory usage: 292.7+ KB


From spanish and nl date to YYYY-MM-DD

In [ ]:

date_string = 'martes 5 de noviembre de 2024'
date_object = dateparser.parse(date_string)
formatted_date = date_object.strftime('%Y-%m-%d')
print(formatted_date)  # Output: 2024-10-31


2024-11-05


In [24]:
sub_data = data.head(20)

sub_data['standard_date'] = sub_data['date'].apply(lambda x: dateparser.parse(x).strftime('%Y-%m-%d'))

print(sub_data[['date', 'standard_date']])

                               date standard_date
0      jueves 31 de octubre de 2024    2024-10-31
1       lunes 28 de octubre de 2024    2024-10-28
2     martes 5 de noviembre de 2024    2024-11-05
3       lunes 28 de octubre de 2024    2024-10-28
4     martes 5 de noviembre de 2024    2024-11-05
5     sábado 2 de noviembre de 2024    2024-11-02
6     sábado 2 de noviembre de 2024    2024-11-02
7     sábado 2 de noviembre de 2024    2024-11-02
8      jueves 31 de octubre de 2024    2024-10-31
9    domingo 3 de noviembre de 2024    2024-11-03
10    domingo 27 de octubre de 2024    2024-10-27
11     martes 22 de octubre de 2024    2024-10-22
12     jueves 24 de octubre de 2024    2024-10-24
13  miércoles 23 de octubre de 2024    2024-10-23
14     jueves 24 de octubre de 2024    2024-10-24
15      lunes 21 de octubre de 2024    2024-10-21
16    domingo 27 de octubre de 2024    2024-10-27
17      lunes 21 de octubre de 2024    2024-10-21
18      lunes 21 de octubre de 2024    2024-10-21


/tmp/ipykernel_280250/1307123335.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_data['standard_date'] = sub_data['date'].apply(lambda x: dateparser.parse(x).strftime('%Y-%m-%d'))


Extract the title

In [26]:

sub_data['title'] = sub_data['article'].apply(lambda x: re.search(r"(.+?):", x).group(1))

# print the title column completely withou truncation
pd.set_option('display.max_colwidth', None)
print(sub_data['title'])


0         biker sobrevive a peligrosa maniobra en la mexico-puebla, pero ve morir a su acompañante
1             arrestan a mujer en la vcarranza acusada de matar a un menor en un hecho de transito
2                   detienen a catorce personas luego de tres dias de operativos en alvaro obregon
3     vinculan a proceso a ricardo "n" ex funcionario de la suprema corte de justicia de la nacion
4                                           caen dos con tarjetas de credito presuntamente robadas
5             policias auxilian a mujer durante una crisis nerviosa en calles del centro historico
6                       acribillan a cinco amigos; uno murio y cuatro mas estan heridos, en la gam
7                                     policia investiga la muerte de un hombre en forum buenavista
8                                                 cae "el vaca" tras robo a mano armada en polanco
9                                               ultiman a balazos a mujer en "el hoyo", iztapalapa
10        

/tmp/ipykernel_280250/1127950711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_data['title'] = sub_data['article'].apply(lambda x: re.search(r"(.+?):", x).group(1))
